# Ransomware Behavior Indicator

**Mass deletion and rename/encryption extensions — stop spread fast.**

This notebook is a **ready-to-use SOC playbook** with:

- Triage checklist
- Splunk SPL queries
- Microsoft Sentinel KQL queries
- Elastic KQL/EQL concepts
- Containment & response commands (PowerShell / Windows / MDE)

> Replace placeholders like `<USER>`, `<HOST>`, `<DEVICE_ID>`, `<TENANT_DOMAIN>` before running.


## Triage Checklist

**Collect:** user/account, host/device, source IP, time window, impacted resources, and any correlated alerts.

**Validate:** baseline/allowlists (VPN egress, jump hosts, admin tools) before escalating.

**Preserve:** keep a copy of key logs and process trees before remediation when possible.

Focus: **process responsible**, **shares impacted**, **extension changes**, **privilege context**.


## Investigation Queries

Check deletion/rename rates and identify initiating process.


In [ ]:
// KQL
let window=10m;
DeviceFileEvents
| where TimeGenerated > ago(7d)
| where DeviceName == '<HOST>'
| where ActionType in~ ('FileDeleted','FileRenamed')
| summarize Deletes=countif(ActionType=='FileDeleted'), Renames=countif(ActionType=='FileRenamed') by InitiatingProcessFileName, InitiatingProcessAccountName, bin(TimeGenerated, window)
| order by Deletes desc


In [ ]:
# Splunk SPL
index=endpoint (file_action=delete OR file_action=rename)
| bucket _time span=10m
| stats count by host user process_name _time
| where count>300


In [ ]:
# PowerShell
## Emergency containment (local)
# Disconnect network quickly (choose one):
# Disable-NetAdapter -Name "Ethernet" -Confirm:$false
# netsh interface set interface "Ethernet" admin=disabled


## Containment & Response

Isolate host(s), disable compromised account, and block SMB to prevent propagation.


In [ ]:
# PowerShell
## Disable SMB inbound temporarily (host-level)
Set-NetFirewallRule -DisplayGroup 'File and Printer Sharing' -Enabled False

## Disable user if compromised (AD)
Disable-ADAccount -Identity <USER>


In [ ]:
# PowerShell
## Preserve evidence
wevtutil epl Security C:\IR\Security.evtx
wevtutil epl Microsoft-Windows-Sysmon/Operational C:\IR\Sysmon.evtx
